In [185]:
import numpy as np
import xmlTranslator as xmlT
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2
import importlib
from utils import get_project_root
from ipywidgets import FloatProgress

TODO: 
- Repeat the sequences 3 times
- Clean all vocabulary into a concise repertoire 
- Add the maj nature
- Extend all chords into 18 enharmonic
- Split into train and test
- create the new MIDI array 
- Pass the MIDI into Embeddings

In [246]:
directory = '/data/iRealXML'
root = get_project_root()
myPath = str(root) + str(directory)

theChordDataset, theOffsetDataset, all_meta = xmlT.parse_info_from_XML(myPath)
print(theChordDataset.shape, theOffsetDataset.shape, len(all_meta))

  0%|          | 0/4007 [00:00<?, ?it/s]

(4006,) (4006,)
(4006,) (4006,) 4006


In [247]:
importlib.reload(xmlT)

id = 0
expand_form_dataset = []
status = True
for i in tqdm(range(theChordDataset.shape[0])):
    song_structure, status = xmlT.expand_song_structure(theChordDataset[i], i)
    if status == False:
        print('Song:', all_meta[i])
        id = i
        break
    song_structure = np.array(song_structure)
    expand_form_dataset.append(song_structure)


  0%|          | 0/4006 [00:00<?, ?it/s]

100%|██████████| 4006/4006 [00:00<00:00, 4066.39it/s]


In [221]:
#check the length of the song

max = {'v': 0, 'id': 0}
min = {'v': 1000, 'id': 0}

for i in range(len(expand_form_dataset)):
    if len(expand_form_dataset[i]) > max['v']:
        max['v'] = len(expand_form_dataset[i])
        max['id'] = i
    if len(expand_form_dataset[i]) < min['v']:
        min['v'] = len(expand_form_dataset[i])
        min['id'] = i

print('Max:', max, 'Min:', min)


Max: {'v': 1321, 'id': 141} Min: {'v': 30, 'id': 492}


In [248]:
importlib.reload(xmlT)

#correct the wrong chords
corrected_sequence = xmlT.repleaceTheseChords(expand_form_dataset, False)

  0%|          | 0/4006 [00:00<?, ?it/s]

In [249]:
#Repeat the sequences 3 times
concatenated_array = []

concatenated_array = np.hstack(corrected_sequence) 

vocab = set(concatenated_array)

print(len(vocab))
print(vocab)

290
{'', 'Pop-Shuffle', 'alter b5 add 9', 'Shuffle Blues', 'add b6 add #9', 'm7 add9', 'Gypsy Waltz', 'Gospel Ballad', 'Form_B', 'Repeat_2', 'maj add 6', 'E', 'add #11 add 7', 'alter b5 add 7', 'dom7', 'add #11 alter b5 add #9', 'Maxixe', 'Soul Ballad', 'Form_Coda', 'Slow Blues', 'dom7 add 13', 'add 9 add b13', 'add b6', 'Bolero', 'Bolero-Son', 'Up Swing', 'Rock Ballad', 'add #9 add b9 alter b5', 'Form_A', 'add #7 add b9', '.', 'add b13', 'Medium Waltz', 'Shuffle', 'sus7 add #11', 'add b9 add #11', 'add #11', 'Repeat_1', 'alter b5 add b13', 'Latin-Swing', 'maj7', ':|', 'add 7 add #9', 'Slow Bossa', 'sus4 add 9', 'alter #11', 'Merengue', 'Slow Shuffle', 'maj add 6 add 9', 'March', '7 add b9', 'Funk', 'Repeat_3', 'aug7', 'Up Tempo', 'alter #5 add #9', 'alter #5 add b13', 'Dreamlike', 'Ballad', 'alter b9 add b13', 'Even 8ths', 'Bossa Nova', 'Medium Country', 'Foxtrot', 'Cb', 'sus2', 'Calypso', 'add b13 add 7', 'add 7 add 9', 'm6', 'Cha Cha', 'sus7 add 9', 'add #11 alter b5', 'add #7', 'Ma

In [279]:
#Add the maj token to the chords
import voicing as vc
voicing = vc.Voicing()
#add the maj token to the chords
new_sequence = None
new_sequence = voicing.add_maj_token(corrected_sequence)

print(new_sequence[2000])

#split any add or alter into a new element in the sequence, the value next to add or alter need to be preserved.



['<style>', 'Slow Swing', '|', 'Form_A', '.', 'Ab', 'maj', '.', 'Ab', 'dom7', '/', 'C', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj add 6', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Bb', 'm7', '.', 'Eb', 'dom7', '.', 'Ab', 'maj add 6', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Db', 'maj', '.', 'D', 'maj', '|', '.', 'Db', 'maj', '.', 'D', 'maj', '.', 'Ab', 'maj', '/', 'E', '.', 'F', 'm7', '|', '.', 'Bb', 'm7', '.', 'Eb', 'dom7', '.', 'Ab', 'maj add 6', 'Form_A', '.', 'Ab', 'maj', '.', 'Ab', 'dom7', '/', 'C', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj add 6', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Bb', 'm7', '.', 'Eb', 'dom7', '.', 'Ab', 'maj add 6', '|', '.', 'Db', 'maj', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Db', 'maj', '.', 'D', 'maj', '|', '.', 'Db', 'maj', '.', 'D', 'maj', '.', 'Ab', 'maj', '/', 'E', '.', 'F', 'm7', '|', '.', 'Bb', 'm7', '.', 'Eb', 'dom7', '.', 'Ab', 'maj add 6', '|', '|', 'Form_B', '.', 'C', 'm', '.', 'C', 'm',

In [281]:
#fix the format of extensions to avoid repeated elements
natures = {'maj', 'm', 'm6', 'm7', 'dom_7', 'maj7', 'o7', 'o', 'sus', 'sus2', 'sus7', 'ø7', 'power', 'm_maj7'}

#subSet = new_sequence[299:300]

for song in tqdm(new_sequence):
    for i, element in enumerate(song):
        if 'add' in element or 'alter' in element:
            #print(element)
            split = element.split(' ')
            size = len(split)
            if size > 2:
                if size%2 == 1:
                    #print('odd', split)
                    nature = split[0]
                    song[i] = nature
                    for n in range(1, size-1):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+n, ext)
                if size%2 == 0:
                    #delete the element
                    song.pop(i)
                    for n in range(0, size-1):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+n, ext)
                    


100%|██████████| 1/1 [00:00<00:00, 2473.06it/s]

[['<style>', 'Samba', '|', '.', 'A', 'dom7', 'add b9', '|', 'Form_A', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'D', 'maj', 'add 6', '|', '.', 'B', 'dom7', 'add b9', 'b9 alter', 'alter #5', '|', '.', 'B', 'dom7', 'add b9', '|', 'Form_B', '.', 'E', 'm7', 'add 9', '|', '.', 'A', 'dom7', 'add 9', '|', '.', 'E', 'm7', 'add 9', '|', '.', 'A', 'dom7', 'add 9', '|', '.', 'E', 'm7', 'add 9', '|', '.', 'A', 'dom7', 'add 9', '|', '.', 'E', 'm7', 'add 9', '|', '.', 'A', 'dom7', 'add 9', '|', '.', 'D', 'maj7', 'add 9', '|', '.', 'D', 'maj7', 'add 9', '|', '.', 'E', 'm7', '|', '.', 'A', 'dom7', 'add 9', '|', '.', 'D', 'maj7', 'add 9', '|', '.', 'D', 'maj7', 'add 9', '|', '.', 'E', 'm7', '|', '.', 'A', 'dom7', 'add 9', '|', 'Form_C', '.', '

In [269]:
#find 
id = 0
find = "add b9 alter #5"

for i in range(len(new_sequence)):
    if find in new_sequence[i]:
        print(i)
        print(new_sequence[i])
        id = i
        break
    
print(all_meta[id])


299
['<style>', 'Samba', '|', '.', 'A', 'dom7', 'add b9', '|', 'Form_A', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'm6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'D', 'maj add 6', '|', '.', 'B', 'dom7', 'add b9 alter #5', '|', '.', 'B', 'dom7', 'add b9', '|', 'Form_B', '.', 'E', 'm7 add 9', '|', '.', 'A', 'dom7 add 9', '|', '.', 'E', 'm7 add 9', '|', '.', 'A', 'dom7 add 9', '|', '.', 'E', 'm7 add 9', '|', '.', 'A', 'dom7 add 9', '|', '.', 'E', 'm7 add 9', '|', '.', 'A', 'dom7 add 9', '|', '.', 'D', 'maj7 add 9', '|', '.', 'D', 'maj7 add 9', '|', '.', 'E', 'm7', '|', '.', 'A', 'dom7 add 9', '|', '.', 'D', 'maj7 add 9', '|', '.', 'D', 'maj7 add 9', '|', '.', 'E', 'm7', '|', '.', 'A', 'dom7 add 9', '|', 'Form_C', '.', 'D', 'maj7', '|', '.', 'D', 'maj7', '|', '.', 'D', 'maj7', '|', '.', 'D', 'maj7

In [216]:
#find duplicated add or alter elements and remove them

sub_group = new_sequence[0:100]
structural_elements = {'.', '|', ':|', '|:', '/', 'N.C.'}

for song in sub_group:
    for i, e in enumerate(song):
        check_extensions = []
        if '.' in e:
            search = True
            stepper = 2
            while search:
                if (stepper + i) >= len(song):
                    search = False
                    #print('End of song')
                    break
                elif i+stepper < len(song) - 1:
                    element = song[i+stepper]
        
                if element in structural_elements or element.startswith('Form_'):
                    search = False
                else:
                    #print(element, i+stepper)
                    check_extensions.append(element)
                    stepper += 1
            if len(check_extensions) > 2:
                print(check_extensions, i)

['dom7', 'add b9 alter #5', 'alter #5', 'add b9', 'alter #5', 'add b9'] 33
['dom7', 'add b9 alter #5', 'alter #5', 'add b9', 'alter #5', 'add b9'] 115
['dom7', 'add b9 alter #5', 'alter #5', 'add b9', 'alter #5', 'add b9'] 262
['m7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 17
['dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 25
['m7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 63
['dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 71
['m7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 145
['dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 153
['m7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 118
['dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 126
['m7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 138
['dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5'] 146
['m7', 'alter b5 add b9', '

In [217]:
print(sub_group[33])

['<style>', 'Ballad', '|', 'Form_A', '.', 'Bb', 'maj7', '|', '.', 'G', 'm7', '.', 'C', 'dom7', '|', '.', 'A', 'm7', '|', '.', 'D', 'm7', '|', '.', 'G', 'm7', '|', '.', 'C', 'dom7', '|', '.', 'F', 'maj7', '|', '.', 'C', 'm7', '.', 'F', 'dom7', 'Form_A', '.', 'Bb', 'maj7', '|', '.', 'G', 'm7', '.', 'C', 'dom7', '|', '.', 'A', 'm7', '|', '.', 'D', 'm7', '|', '.', 'G', 'm7', '|', '.', 'C', 'dom7', '|', '.', 'F', 'maj', 'add 6', '|', '.', 'C', 'm7', '.', 'F', 'dom7', '|', 'Form_B', '.', 'Bb', 'maj7', '.', 'Eb', 'dom7', 'add 9', '|', '.', 'A', 'm7', '.', 'D', 'm7', '|', '.', 'G', 'm7', '.', 'C', 'dom7', '|', '.', 'F', 'maj7', '|', '.', 'E', 'm7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5', '.', 'A', 'dom7', 'alter b5 add b9', 'add b9', 'alter b5', 'add b9', 'alter b5', '|', '.', 'D', 'm7', '|', '.', 'G', 'dom7', '|', '.', 'C', 'dom7', '.', 'C', 'm7', '.', 'F', 'dom7', '|', 'Form_A', '.', 'Bb', 'maj7', '|', '.', 'G', 'm7', '.', 'C', 'dom7', '|', '.', 'A', 'm7', '|', '.', 'D